<a href="https://colab.research.google.com/github/seifahmed2004/Information-retrieval-project-/blob/main/IR_project_seif_ahmed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install python-terrier --quiet
!pip install nltk --quiet
!pip install --ignore-installed blinker
!pip install git+https://github.com/experimaestro/experimaestro-ir.git --quiet
!pip install transformers
!pip install flair
!git clone https://github.com/terrierteam/terrier-prf/
!apt-get install maven   #used for Java projects to manage project dependencies and build processes
%cd /content/terrier-prf/
!mvn install
!pwd
%cd ..

  Using cached blinker-1.8.2-py3-none-any.whl (9.5 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
fatal: destination path 'terrier-prf' already exists and is not an empty directory.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
maven is already the newest version (3.6.3-5).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
/content/terrier-prf
[INFO] Scanning for projects...
[INFO] 
[INFO] ----------------------< org.terrier:terrier-prf >-----------------------
[INFO] Building terrier-prf 0.2-SNAPSHOT
[INFO] --------------------------------[ jar ]---------------------------------
[INFO] 
[INFO] --- maven-resources-plugin:2.6:resources (default-resources) @ terrier-prf ---
[INFO] Using 'UTF-8' encoding to copy filtered resources.
[INFO] skip non existing resourceDirectory /content/terrier-prf/src/main/resources
[INFO] 
[INFO] --- ma

In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import *
from nltk.stem.porter import *
from flair.data import Sentence
from flair.embeddings import WordEmbeddings
from transformers import AutoTokenizer, AutoModel
import tensorflow as tf
import tensorflow_hub as hub
import gensim
from gensim.models import Word2Vec
pd.set_option('display.max_colwidth', 150)
import pyterrier as pt
if not pt.started():
  # In this lab, we need to specify that we start PyTerrier with PRF enabled
  pt.init(boot_packages=["com.github.terrierteam:terrier-prf:-SNAPSHOT"])

PyTerrier 0.10.1 has loaded Terrier 5.9 (built by craigm on 2024-05-02 17:40) and terrier-helper 0.0.8



In [3]:
data = pd.read_csv("/content/emotion_sentimen_dataset (1).csv")
data

,ID,text,Emotion
0,0,i seriously hate one subject to death but now i feel reluctant to drop it,hate
1,1,im so full of life i feel appalled,neutral
2,2,i sit here to write i start to dig out my feelings and i think that i am afraid to accept the possibility that he might not make it,neutral
3,3,ive been really angry with r and i feel like an idiot for trusting him in the first place,anger
4,4,i feel suspicious if there is no one outside like the rapture has happened or something,neutral
...,...,...,...
839550,839550,i feel like telling these horny devils to find a site more suited to that sort of interest the playboy if there is one forum perhaps,neutral
839551,839551,i began to realize that when i was feeling agitated or restless that i would have a thought to go do the dishes,neutral
839552,839552,i feel very curious be why previous early dawn in the time here don t you seek his trouble,neutral
839553,839553,i feel that becuase of the tyranical nature of the government in el salvador and the savage social conditions of the general people that the set o...,neutral


In [4]:
nltk.download('stopwords')
print(stopwords.words('english'))
nltk.download('stopwords')
nltk.download('punkt')

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
stop_words = set(stopwords.words('english'))

In [ ]:
# Function to remove stopwords
def remove_stopwords(text):

    tokens = word_tokenize(text)
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in stop_words] #Lower is used to normalize al the words make them in lower case
    print('Tokens are:',tokens,'\n')
    return ' '.join(filtered_tokens)

# Apply the remove_stopwords function to the 'content' column
data['processed_text'] = data['text'].apply(remove_stopwords)

Streaming output truncated to the last 5000 lines.
Tokens are: ['i', 'am', 'feeling', 'and', 'put', 'it', 'into', 'an', 'action', 'plan', 'that', 'simply', 'can', 'not', 'be', 'ignored'] 

Tokens are: ['i', 'was', 'left', 'feeling', 'a', 'little', 'envious'] 

Tokens are: ['im', 'just', 'less', 'resilient', 'than', 'when', 'i', 'was', 'younger', 'but', 'the', 'events', 'of', 'the', 'past', 'few', 'months', 'have', 'left', 'me', 'feeling', 'bitter'] 

Tokens are: ['i', 'feel', 'this', 'less', 'successful', 'as', 'a', 'piece', 'maybe', 'a', 'little', 'over', 'board', 'with', 'choices', 'of', 'materials'] 

Tokens are: ['i', 'am', 'feeling', 'rather', 'lousy', 'again'] 

Tokens are: ['i', 'get', 'a', 'wonderful', 'idea', 'something', 'i', 'feel', 'passionate', 'about', 'i', 'get', 'excited', 'and', 'then', 'poof', 'i', 'manage', 'to', 'put', 'myself', 'into', 'a', 'situation', 'where', 'i', 'procrastinate'] 

Tokens are: ['i', 'am', 'feeling', 'restless', 'i', 'go', 'for', 'a', 'walk', 'o

In [ ]:
stemmer = PorterStemmer()

In [ ]:
def Steem_text(text):

    tokens = word_tokenize(text)
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(stemmed_tokens)

In [ ]:
#a function to clean the documents
def clean(text):
   text = re.sub(r"http\S+", " ", text) # remove urls
   text = re.sub(r"RT ", " ", text) # remove rt
   text = re.sub(r"@[\w]*", " ", text) # remove handles
   text = re.sub(r"[\.\,\#_\|\:\?\?\/\=]", " ", text) # remove special characters
   text = re.sub(r'\t', ' ', text) # remove tabs
   text = re.sub(r'\n', ' ', text) # remove line jump
   text = re.sub(r"\s+", " ", text) # remove extra white space
   text = text.strip()
   return text
data['processed_text'] = data['processed_text'].apply(clean)
data

In [ ]:
data['processed_text']= data['processed_text'].apply(Steem_text)
data

In [ ]:
data['docno'] = data['ID'].astype(str)
data

In [ ]:
indexer = pt.DFIndexer("./DatasetIndex", overwrite=True)
index_ref = indexer.index(data["processed_text"], data["docno"])
print(index_ref.toString())

In [ ]:
#we will first load the index
index = pt.IndexFactory.of(index_ref)
#we will call getCollectionStatistics() to check the stats
print(index.getCollectionStatistics().toString())

In [ ]:
for kv in index.getLexicon():
  print("%s -> %s " % (kv.getKey(), kv.getValue().toString()))

In [ ]:
index.getLexicon()["result"].getDocumentFrequency()

In [ ]:
x = index.getDirectIndex()
y = index.getDocumentIndex()
z = index.getLexicon()
a = 0
for posting in x.getPostings(y.getDocumentEntry(a)):
    termid = posting.getId()
    B = z.getLexiconEntry(termid)
    print("%s with frequency %d" % (B.getKey(),posting.getFrequency()))

In [ ]:
def preprocess(sentence):
    sentence = remove_stopwords(sentence)
    sentence = clean(sentence)
    sentence = Steem_text(sentence)
    return sentence

In [ ]:
# Preprocess user query
query = "fun"
query = preprocess(query)

In [ ]:
# Set up retrieval model with TF-IDF as weighting model
tfidf_retr = pt.BatchRetrieve(index, controls={"wmodel": "TF_IDF"}, num_results=10)

# Search for relevant documents using the retrieval model
results = tfidf_retr.search(query)
results

In [ ]:
# Retrieve top relevant documents based on the query
top_answer = data[data["docno"].isin(['6419', '48097'])]['processed_text']
top_answer

In [ ]:
# Calculate frequency of the term 'empty' in the top related documents
freq_of_items = []
for i in range(len(top_answer)):
    tweet = top_answer.iloc[i].split()
    counter = 0
    for word_in_tweet in tweet:
        if word_in_tweet == 'fun':
            counter += 1
    freq_of_items.append(counter)
print('The frequency of the item in the top related docs are:', freq_of_items)

In [ ]:
index = pt.IndexFactory.of(index_ref)

In [ ]:
# Define our retrieval model
bm25 = pt.BatchRetrieve(index, wmodel="BM25",num_results=10)
results = bm25.search(query)
results

In [ ]:
data[['text']][data['docno'].isin(results['docno'].loc[0:4].tolist())]

In [ ]:
rm3_expander = pt.rewrite.RM3(index, fb_terms=10, fb_docs=100)

rm3_qe = bm25 >> rm3_expander
expanded_query = rm3_qe.search(query).iloc[0]["query"]

expanded_query

In [ ]:
# Just print the expanded query with term scores
for s in expanded_query.split()[1:]:
  print(s)

print("\n" + query)

In [ ]:
# After that you can search using the expanded query
expanded_query_formatted = ' '.join(expanded_query.split()[1:])

results_wqe = bm25.search(expanded_query_formatted)

print("   Before Expansion    After Expansion")
print(pd.concat([results[['docid','score']][0:5].add_suffix('_1'),
            results_wqe[['docid','score']][0:5].add_suffix('_2')], axis=1).fillna(''))


#Let's check the tweets text for the top 5 retrieved tweets
data[['text']][data['docno'].isin(results_wqe['docno'].loc[0:5].tolist())]

In [ ]:
!pip install flask_ngrok
!pip install pyngrok

In [ ]:
df2 = data.head(50)

df2 = df2.to_dict()

df2

In [ ]:
def sui(df2 , que):
 i = 0

 quer = preprocess(que)

 docs_id = []

 for key, value in df2.items():
   if key == 'processed_text':
         val = value.values()
         for doc in val:
           terms = doc.split()
           for term in terms:
             if term == quer and i not in docs_id:
               docs_id.append(f'''Document number {i} -----> \n{data["text"][i]}''')
           i = i + 1
 return docs_id

In [ ]:
query2 = "like"
X = sui(df2 , query2)
X

In [ ]:
from google.colab.output import eval_js
print (eval_js("google.colab.kernel.proxyPort(5000)"))

In [ ]:
from flask import Flask, request
from flask_ngrok import run_with_ngrok

# Assuming you've already defined the sui function and imported necessary modules

app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
    return """
    <style>
        body {
            background-color: 	#f1c232;
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
        }

        .header {
            background-color:		#f1c232;
            color: black ;
            padding: 20px 0;

        }

        .container {
            text-align: center;
        }

        h1 {
            text-align: center;
            margin:0;
            padding: 10px 0;
        }

        #searchInput {
            padding: 10px;
            border: 1px solid #ccc;
            border-radius: 50px; /* Increased border-radius for a rounded appearance */
            margin-bottom: 10px;
            width: 300px; /* Adjust the width as needed */
            box-sizing: border-box; /* Include padding and border in the element's total width */
            transition: border-color 0.5s; /* Smooth transition for border color change */
        }

        #searchInput:focus {
            border-color: #403072; /* Change border color on focus */
        }

        button {
            padding: 10px 20px;
            background-color:#ce7e00;
            color: white;
            border: none;
            border-radius: 20px; /* Increased border-radius for a rounded appearance */
            cursor: pointer;
            transition: background-color 0.5s; /* Smooth transition for background color change */
        }

        button:hover {
            background-color: #7d73af; /* Change background color on hover */
        }

          #searchResult {
            text-align: 10 px; /* Center the search results */
            color: black; /* Set the color of search results */
            margin: 10 px;
            padding: 10px
        }
    </style>

    <div class="header">
        <h1>Search Engine</h1>
    </div>
    <div class="container">
        <input type="text" id="searchInput" placeholder="search...">
        <button onclick="search()">Search</button>
    </div>
    <div id="searchResult"></div>

    <script>
        function search() {
            var searchTerm = document.getElementById("searchInput").value;
            fetch('/search', {
                method: 'POST',
                body: JSON.stringify({ query: searchTerm }),
                headers:{
                    'Content-Type': 'application/json'
                }
            })
            .then(response => response.json())
            .then(data => {
                console.log("Received data:", data); // Debug: Check if data is received
                var resultDiv = document.getElementById("searchResult");
                resultDiv.innerHTML = "<h2>Relevant Documents IDs:</h2>";
                if (data.results.length === 0) {
                    resultDiv.innerHTML += "<p>No documents found</p>";
                } else {
                    data.results.forEach(doc => {
                        console.log("Displaying document:", doc); // Debug: Check if document is displayed
                        resultDiv.innerHTML += "<p>" + doc + "</p>";
                    });
                }
            })
            .catch(error => {
                console.error('Error occurred during fetch:', error); // Debug: Log fetch errors
            });
        }
    </script>
    """

@app.route("/search", methods=['POST'])
def search():
    query = request.json['query']
    print("Received query:", query)  # Debug: Check if Flask receives the query
    results = sui(df2, query)
    print("Search results:", results)  # Debug: Check if sui function returns results
    return {'results': results}

app.run()


In [ ]:
vaswani_dataset = pt.datasets.get_dataset("vaswani")

df= vaswani_dataset.get_topics()

data['docno'] = df.index

# Rename column 'A' to 'X'
df = df.rename(columns={'query': 'Text'})

qrels = vaswani_dataset.get_qrels()

qrels['docno']=qrels['docno'].astype(str)

df

In [ ]:
indexref2 = vaswani_dataset.get_index()
index2 = pt.IndexFactory.of(indexref2)

print(index2.getCollectionStatistics().toString())

In [ ]:
retr = pt.BatchRetrieve(index2, controls = {"wmodel": "TF_IDF"})

res = retr.search("mathematical")
res

In [ ]:
eval = pt.Evaluate(res,qrels)
eval